In [103]:
import pandas as pd
import numpy as np
import seaborn as sns

# plotting 환경 설정
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
plt.rcParams['figure.figsize'] = (5,5) # 그래프 크기
plt.rcParams['font.family'] = 'Malgun Gothic' # 글꼴
plt.rcParams['font.size'] = 12 # 글꼴 크기
plt.rcParams['axes.unicode_minus'] = False # 마이너스 표기

# 그래프가 노트북 안에 보이게 하기 위해
%matplotlib inline

In [104]:
# 데이터를 구분하는 문자를 지정하는 옵션
vod8 = pd.read_csv('../data/데이터스쿨3차_2308월/데이터스쿨_3차_VOD_2308.csv', encoding = 'cp949', sep = '\t')
vod9 = pd.read_csv('../data/데이터스쿨3차_2309월/데이터스쿨_3차_VOD_2309.csv', encoding = 'cp949', sep = '\t')
con8 = pd.read_csv('../data/데이터스쿨3차_2308월/데이터스쿨_3차_콘텐츠_2308.csv', encoding = 'cp949', sep = '\t')
con9 = pd.read_csv('../data/데이터스쿨3차_2309월/데이터스쿨_3차_콘텐츠_2309.csv', encoding = 'cp949', sep = '\t')

In [105]:
v8 = vod8.copy()
v9 = vod9.copy()
c8 = con8.copy()
c9 = con9.copy()

In [106]:
v8[v8['asset_nm'].str.contains('다이노코어')]

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
4025,64944000,(더빙)다이노코어 시즌1. 01회,TV애니메이션,액션/모험,844,출동! 다이노 코어. 피자 배달을 하던 렉스가 우연히 다이노코어들을 만나 지구를 지...,"렉스,에밀리,존,빌,파브로,다크노",0:15,20230804130141


In [107]:
con9.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14387 entries, 0 to 14386
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subsr           14387 non-null  int64 
 1   series_nm       6458 non-null   object
 2   super_asset_nm  14387 non-null  object
 3   ct_cl           14387 non-null  object
 4   genre_of_ct_cl  14387 non-null  object
 5   SMRY            14387 non-null  object
 6   ACTR_DISP       14387 non-null  object
 7   disp_rtm        13260 non-null  object
 8   log_dt          14387 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 1011.7+ KB


# content 전처리

In [108]:
# con 데이터프레임을 header가 같은 열을 기준으로 합치기
con89 = pd.concat([con8, con9], ignore_index=True)
con89.tail()

,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,SMRY,ACTR_DISP,disp_rtm,log_dt
30059,64551000,NaN,슈퍼 토끼,키즈,학습,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내...",-,0:12,20230930121656
30060,64551000,NaN,슈퍼 토끼,키즈,학습,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내...",-,0:12,20230930121646
30061,64551000,NaN,슈퍼 토끼,키즈,학습,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내...",-,0:12,20230930121655
30062,64551000,NaN,슈퍼 토끼,키즈,학습,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내...",-,0:12,20230930121655
30063,64551000,NaN,슈퍼 토끼,키즈,학습,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내...",-,0:12,20230930121654


In [109]:
# 예고가 위치한 컬럼 확인
con89['super_asset_nm'].str.contains("사전구매").value_counts()

super_asset_nm
False    30023
True        41
Name: count, dtype: int64

In [110]:
from pandas._libs.tslibs.offsets import Hour
import re
def preprocessing_con(df):
  # 특수문자 정리
  # 예고편 제거
  df = df[~df['super_asset_nm'].str.contains(r'\(예고편\)|\(예고\)', regex=True)]

  # series_nm 괄호와 그 안의 내용 제거
  df['series_nm'] = df['series_nm'].str.replace(r'\([^()]*\)', '', regex=True)
  df['series_nm'] = df['series_nm'].str.replace(r'\[[^\[\]]*\]', '', regex=True)
  df['series_nm'] = df['series_nm'].str.replace(r'\.\.\.', '', regex=True)
  df['series_nm'] = df['series_nm'].str.replace('-', ' ')
  df['series_nm'] = df['series_nm'].str.rstrip('.')

  # super_asset_nm 변경
  df['super_asset_nm'] = df['super_asset_nm'].str.replace(r'\([^()]*\)', '', regex=True)
  df['super_asset_nm'] = df['super_asset_nm'].str.replace(r'\[[^\[\]]*\]', '', regex=True)
  df['super_asset_nm'] = df['super_asset_nm'].str.replace(r'\.\.\.', '', regex=True)
  df['super_asset_nm'] = df['super_asset_nm'].str.replace('-', ' ')
  df['super_asset_nm'] = df['super_asset_nm'].str.rstrip('.')

  # 회, 회., 화, 실사판, 무삭제 제거
  df['series_nm'] = df['series_nm'].str.replace(r'무삭제', '', regex = True)
  df['series_nm'] = df['series_nm'].str.replace(r'실사판', '', regex = True)
  df['series_nm'] = df['series_nm'].str.replace(r'\d+회$', '', regex=True)
  df['series_nm'] = df['series_nm'].str.replace(r'\d+회\.', '', regex=True)
  df['series_nm'] = df['series_nm'].str.replace(r'\d+화$', '', regex = True)
  df['series_nm'] = df['series_nm'].str.strip()
  df['series_nm'] = df['series_nm'].str.rstrip('.')

  df['super_asset_nm'] = df['super_asset_nm'].str.replace(r'무삭제', '', regex = True)
  df['super_asset_nm'] = df['super_asset_nm'].str.replace(r'실사판', '', regex = True)
  df['super_asset_nm'] = df['super_asset_nm'].str.replace(r'\d+회$', '', regex=True)
  df['super_asset_nm'] = df['super_asset_nm'].str.replace(r'\d+회\.', '', regex=True)
  df['super_asset_nm'] = df['super_asset_nm'].str.replace(r'\d+화$', '', regex = True)
  df['super_asset_nm'] = df['super_asset_nm'].str.strip()
  df['super_asset_nm'] = df['super_asset_nm'].str.rstrip('.')

  # 'series_nm'이 null인 경우, 'super_asset_nm'의 값으로 대체
  df['series_nm'].fillna(df['super_asset_nm'], inplace=True)

  # disp_rtm 문자열을 분 단위로 변경
  def convert_runtime(runtime_str):
    # 입력값이 NaN이면 0 반환
    if pd.isna(runtime_str):
        return 0

    hours = int(runtime_str.split(':')[0])
    minutes = int(runtime_str.split(':')[1])
    total_minutes = hours * 60 + minutes
    total_minutes = str(total_minutes) + '분'
    return total_minutes

  df['disp_rtm'] = df['disp_rtm'].apply(convert_runtime)

  return df

In [111]:
con_89 = preprocessing_con(con89)
con_89.head()

C:\Users\USER\AppData\Local\Temp\ipykernel_3024\2380850214.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['series_nm'] = df['series_nm'].str.replace(r'\([^()]*\)', '', regex=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_3024\2380850214.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['series_nm'] = df['series_nm'].str.replace(r'\[[^\[\]]*\]', '', regex=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_3024\2380850214.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,SMRY,ACTR_DISP,disp_rtm,log_dt
0,65968000,핑크퐁 동물 동요,핑크퐁 동물 동요,키즈,기타,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",핑크퐁,1분,20230813071155
1,65968000,핑크퐁 동물 동요,핑크퐁 동물 동요,키즈,기타,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",핑크퐁,1분,20230813071215
2,66056000,라바 키즈동요 Part4,라바 키즈동요 Part4,키즈,기타,징글벨 크리스마스. 안녕하신가 친구들! 메리 크리스마스~ 징글벨 크리스마스 징글벨 ...,"엘로우,레드",2분,20230808085440
3,66056000,라바 키즈동요 Part4,라바 키즈동요 Part4,키즈,기타,징글벨 크리스마스. 안녕하신가 친구들! 메리 크리스마스~ 징글벨 크리스마스 징글벨 ...,"엘로우,레드",2분,20230808085432
4,63086000,지니강이 플러스 시즌7,지니강이 플러스 시즌7,키즈,기타,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...,"헤이지니,럭키강이",11분,20230827145542


In [112]:
# 확인
con_89[con_89["series_nm"].isnull()]

,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,SMRY,ACTR_DISP,disp_rtm,log_dt


In [113]:
# 새로운 변수 추가
new_column_name  = 'summary'
con_89["summary"] = pd.Series(dtype='object')

new_column_name  = 'director'
con_89["director"] = pd.Series(dtype='object')

new_column_name  = 'category'
con_89["category"] = pd.Series(dtype='object')
con_89.head()

C:\Users\USER\AppData\Local\Temp\ipykernel_3024\3982161653.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  con_89["summary"] = pd.Series(dtype='object')
C:\Users\USER\AppData\Local\Temp\ipykernel_3024\3982161653.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  con_89["director"] = pd.Series(dtype='object')
C:\Users\USER\AppData\Local\Temp\ipykernel_3024\3982161653.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,SMRY,ACTR_DISP,disp_rtm,log_dt,summary,director,category
0,65968000,핑크퐁 동물 동요,핑크퐁 동물 동요,키즈,기타,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",핑크퐁,1분,20230813071155,NaN,NaN,NaN
1,65968000,핑크퐁 동물 동요,핑크퐁 동물 동요,키즈,기타,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",핑크퐁,1분,20230813071215,NaN,NaN,NaN
2,66056000,라바 키즈동요 Part4,라바 키즈동요 Part4,키즈,기타,징글벨 크리스마스. 안녕하신가 친구들! 메리 크리스마스~ 징글벨 크리스마스 징글벨 ...,"엘로우,레드",2분,20230808085440,NaN,NaN,NaN
3,66056000,라바 키즈동요 Part4,라바 키즈동요 Part4,키즈,기타,징글벨 크리스마스. 안녕하신가 친구들! 메리 크리스마스~ 징글벨 크리스마스 징글벨 ...,"엘로우,레드",2분,20230808085432,NaN,NaN,NaN
4,63086000,지니강이 플러스 시즌7,지니강이 플러스 시즌7,키즈,기타,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...,"헤이지니,럭키강이",11분,20230827145542,NaN,NaN,NaN


In [114]:
new_order = ["series_nm", "summary", "ACTR_DISP", "director", "ct_cl", "genre_of_ct_cl", "category", "disp_rtm", "SMRY"]
con = con_89[new_order]
con.tail()

,series_nm,summary,ACTR_DISP,director,ct_cl,genre_of_ct_cl,category,disp_rtm,SMRY
30059,슈퍼 토끼,NaN,-,NaN,키즈,학습,NaN,12분,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내..."
30060,슈퍼 토끼,NaN,-,NaN,키즈,학습,NaN,12분,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내..."
30061,슈퍼 토끼,NaN,-,NaN,키즈,학습,NaN,12분,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내..."
30062,슈퍼 토끼,NaN,-,NaN,키즈,학습,NaN,12분,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내..."
30063,슈퍼 토끼,NaN,-,NaN,키즈,학습,NaN,12분,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내..."


In [115]:
# 이름 변경
con.columns = ['name', 'Summary', 'actors', 'director', 'BigCategory', 'SmallCategory', 'Category', 'runningtime', 'SMRY']
con.tail()

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY
30059,슈퍼 토끼,NaN,-,NaN,키즈,학습,NaN,12분,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내..."
30060,슈퍼 토끼,NaN,-,NaN,키즈,학습,NaN,12분,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내..."
30061,슈퍼 토끼,NaN,-,NaN,키즈,학습,NaN,12분,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내..."
30062,슈퍼 토끼,NaN,-,NaN,키즈,학습,NaN,12분,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내..."
30063,슈퍼 토끼,NaN,-,NaN,키즈,학습,NaN,12분,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내..."


# vod 전처리

In [116]:
# con 데이터프레임을 header가 같은 열을 기준으로 합치기
vod89 = pd.concat([vod8, vod9], ignore_index=True)
vod89.tail()

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
10652,60169000,간질간질,키즈,학습,360,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...,-,0:06,20230914211110
10653,66317000,알사탕,키즈,학습,480,"인기 작가 백희나 신간. 교보문고, YES24 베스트셀러! 상대방의 입장을 헤아리고...",-,0:08,20230903223544
10654,66317000,알사탕,키즈,학습,200,"인기 작가 백희나 신간. 교보문고, YES24 베스트셀러! 상대방의 입장을 헤아리고...",-,0:08,20230903223210
10655,66900000,월간 아기상어(영어) 01회,키즈,학습,46,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...,아기상어,0:01,20230904120057
10656,60169000,놀자!,키즈,학습,407,[인기작가 박정섭의 그림책] 놀고 싶은 어린이들 모두 모두 모여라! 온몸으로 뛰어놀...,-,0:07,20230914210135


In [118]:
# asset_nm 괄호와 괄호 안의 데이터 제거
def preprocessing_vod(df):
  # 특수문자 정리
  ## 예고편 제거
  df = df[~df['asset_nm'].str.contains(r'\(예고편\)|\(예고\)', regex=True)]

  # series_nm 괄호와 그 안의 내용 제거
  df['asset_nm'] = df['asset_nm'].str.replace(r'\([^()]*\)', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\[[^\[\]]*\]', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\.\.\.', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace('-', ' ')
  df['asset_nm'] = df['asset_nm'].str.rstrip('.')

  # 회, 회., 화, 실사판, 무삭제 제거
  df['asset_nm'] = df['asset_nm'].str.replace(r'실사판', '', regex = True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'무삭제', '', regex = True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\d+회$', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\d+회\.', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\d+화$', '', regex = True)
  df['asset_nm'] = df['asset_nm'].str.strip()
  df['asset_nm'] = df['asset_nm'].str.rstrip('.')

  # disp_rtm 문자열을 분 단위로 변경
  def convert_runtime(runtime_str):
    # 입력값이 NaN이면 0 반환
    if pd.isna(runtime_str):
        return 0

    hours = int(runtime_str.split(':')[0])
    minutes = int(runtime_str.split(':')[1])
    total_minutes = hours * 60 + minutes
    total_minutes = str(total_minutes) + '분'
    return total_minutes

  df['disp_rtm'] = df['disp_rtm'].apply(convert_runtime)

  return df

In [119]:
vod_89 = preprocessing_vod(vod89)
vod_89.head()

C:\Users\USER\AppData\Local\Temp\ipykernel_3024\3131716609.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['asset_nm'] = df['asset_nm'].str.replace(r'\([^()]*\)', '', regex=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_3024\3131716609.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['asset_nm'] = df['asset_nm'].str.replace(r'\[[^\[\]]*\]', '', regex=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_3024\3131716609.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
0,65941000,그것이알고싶다,TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80분,20230812163507
1,66873000,그것이알고싶다,TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80분,20230816205227
2,66873000,그것이알고싶다,TV 시사/교양,기타,718,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80분,20230829194727
3,61689000,그것이알고싶다,TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80분,20230813130609
4,61619000,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,4183,"살인범의 미토콘드리아 - 2006 냉동고 살인사건. 2006년 7월 23일, 서울 ...","장도연,장현성,장성규",73분,20230804092737


In [120]:
new_column_name  = 'summary'
vod_89["summary"] = pd.Series(dtype='object')

new_column_name  = 'director'
vod_89["director"] = pd.Series(dtype='object')

new_column_name  = 'category'
vod_89["category"] = pd.Series(dtype='object')
vod_89.head()

C:\Users\USER\AppData\Local\Temp\ipykernel_3024\3011133651.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vod_89["summary"] = pd.Series(dtype='object')
C:\Users\USER\AppData\Local\Temp\ipykernel_3024\3011133651.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vod_89["director"] = pd.Series(dtype='object')
C:\Users\USER\AppData\Local\Temp\ipykernel_3024\3011133651.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt,summary,director,category
0,65941000,그것이알고싶다,TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80분,20230812163507,NaN,NaN,NaN
1,66873000,그것이알고싶다,TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80분,20230816205227,NaN,NaN,NaN
2,66873000,그것이알고싶다,TV 시사/교양,기타,718,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80분,20230829194727,NaN,NaN,NaN
3,61689000,그것이알고싶다,TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80분,20230813130609,NaN,NaN,NaN
4,61619000,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,4183,"살인범의 미토콘드리아 - 2006 냉동고 살인사건. 2006년 7월 23일, 서울 ...","장도연,장현성,장성규",73분,20230804092737,NaN,NaN,NaN


In [121]:
# 순서 변경
new_order = ["asset_nm", "summary", "ACTR_DISP", "director", "ct_cl", "genre_of_ct_cl", "category", "disp_rtm","SMRY"]
vod = vod_89[new_order]
vod.tail()

,asset_nm,summary,ACTR_DISP,director,ct_cl,genre_of_ct_cl,category,disp_rtm,SMRY
10652,간질간질,NaN,-,NaN,키즈,학습,NaN,6분,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...
10653,알사탕,NaN,-,NaN,키즈,학습,NaN,8분,"인기 작가 백희나 신간. 교보문고, YES24 베스트셀러! 상대방의 입장을 헤아리고..."
10654,알사탕,NaN,-,NaN,키즈,학습,NaN,8분,"인기 작가 백희나 신간. 교보문고, YES24 베스트셀러! 상대방의 입장을 헤아리고..."
10655,월간 아기상어,NaN,아기상어,NaN,키즈,학습,NaN,1분,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...
10656,놀자!,NaN,-,NaN,키즈,학습,NaN,7분,[인기작가 박정섭의 그림책] 놀고 싶은 어린이들 모두 모두 모여라! 온몸으로 뛰어놀...


In [122]:
vod.columns = ['name', 'Summary', 'actors', 'director', 'BigCategory', 'SmallCategory', 'Category', 'runningtime', 'SMRY']
vod.tail()

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY
10652,간질간질,NaN,-,NaN,키즈,학습,NaN,6분,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...
10653,알사탕,NaN,-,NaN,키즈,학습,NaN,8분,"인기 작가 백희나 신간. 교보문고, YES24 베스트셀러! 상대방의 입장을 헤아리고..."
10654,알사탕,NaN,-,NaN,키즈,학습,NaN,8분,"인기 작가 백희나 신간. 교보문고, YES24 베스트셀러! 상대방의 입장을 헤아리고..."
10655,월간 아기상어,NaN,아기상어,NaN,키즈,학습,NaN,1분,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...
10656,놀자!,NaN,-,NaN,키즈,학습,NaN,7분,[인기작가 박정섭의 그림책] 놀고 싶은 어린이들 모두 모두 모여라! 온몸으로 뛰어놀...


In [123]:
vod_data = pd.concat([con, vod], ignore_index=True)
vodd = vod_data.copy()
vod_data.head()

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY
0,핑크퐁 동물 동요,NaN,핑크퐁,NaN,키즈,기타,NaN,1분,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ..."
1,핑크퐁 동물 동요,NaN,핑크퐁,NaN,키즈,기타,NaN,1분,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ..."
2,라바 키즈동요 Part4,NaN,"엘로우,레드",NaN,키즈,기타,NaN,2분,징글벨 크리스마스. 안녕하신가 친구들! 메리 크리스마스~ 징글벨 크리스마스 징글벨 ...
3,라바 키즈동요 Part4,NaN,"엘로우,레드",NaN,키즈,기타,NaN,2분,징글벨 크리스마스. 안녕하신가 친구들! 메리 크리스마스~ 징글벨 크리스마스 징글벨 ...
4,지니강이 플러스 시즌7,NaN,"헤이지니,럭키강이",NaN,키즈,기타,NaN,11분,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...


### 중복 제거

In [124]:
# 'name'과 'BigCategory'를 기준으로 중복된 값 삭제
vod_data = vod_data.drop_duplicates(subset=['name', 'BigCategory'])
vod_data = vod_data.reset_index(drop=True)
vod_data.tail()

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY
4736,엄마가 화났다,NaN,-,NaN,키즈,학습,NaN,8분,인기작가 최숙희의 그림책. 아이들에게 건네는 화해와 위로의 메시지! 아이와 엄마 사...
4737,핑크퐁 자동차 동화,NaN,핑크퐁,NaN,키즈,학습,NaN,4분,Lil Reds First Fire(용감한 소방차 뚜뚜와 뿌뿌). 불이 난 곳이면 ...
4738,간질간질,NaN,-,NaN,키즈,학습,NaN,6분,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...
4739,월간 아기상어,NaN,아기상어,NaN,키즈,학습,NaN,1분,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...
4740,놀자!,NaN,-,NaN,키즈,학습,NaN,7분,[인기작가 박정섭의 그림책] 놀고 싶은 어린이들 모두 모두 모여라! 온몸으로 뛰어놀...


In [125]:
# Category 구분
vod_data['Category'] = np.where(vod_data['BigCategory'] == '영화', '영화', np.where(vod_data['BigCategory'] == '키즈', '키즈', 'TV프로그램'))
vod_data

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY
0,핑크퐁 동물 동요,NaN,핑크퐁,NaN,키즈,기타,키즈,1분,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ..."
1,라바 키즈동요 Part4,NaN,"엘로우,레드",NaN,키즈,기타,키즈,2분,징글벨 크리스마스. 안녕하신가 친구들! 메리 크리스마스~ 징글벨 크리스마스 징글벨 ...
2,지니강이 플러스 시즌7,NaN,"헤이지니,럭키강이",NaN,키즈,기타,키즈,11분,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...
3,엉뚱남매 요리조리쇼,NaN,"기원,예현,홍윤화",NaN,키즈,기타,키즈,11분,먹을 수 있는 물병. 지구 환경을 지키자는 엉뚱남매. 하지만 말처럼 쉽게 지켜지지가...
4,라바 키즈동요 Part3,NaN,-,NaN,키즈,기타,키즈,2분,CLEAN UP SONG. Clean up Clean up everybody cle...
...,...,...,...,...,...,...,...,...,...
4736,엄마가 화났다,NaN,-,NaN,키즈,학습,키즈,8분,인기작가 최숙희의 그림책. 아이들에게 건네는 화해와 위로의 메시지! 아이와 엄마 사...
4737,핑크퐁 자동차 동화,NaN,핑크퐁,NaN,키즈,학습,키즈,4분,Lil Reds First Fire(용감한 소방차 뚜뚜와 뿌뿌). 불이 난 곳이면 ...
4738,간질간질,NaN,-,NaN,키즈,학습,키즈,6분,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...
4739,월간 아기상어,NaN,아기상어,NaN,키즈,학습,키즈,1분,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...


In [126]:
# runningtime에 분 추가 
vod_data['runningtime'] = vod_data["runningtime"].astype(str) + '분'
vod_data

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY
0,핑크퐁 동물 동요,NaN,핑크퐁,NaN,키즈,기타,키즈,1분분,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ..."
1,라바 키즈동요 Part4,NaN,"엘로우,레드",NaN,키즈,기타,키즈,2분분,징글벨 크리스마스. 안녕하신가 친구들! 메리 크리스마스~ 징글벨 크리스마스 징글벨 ...
2,지니강이 플러스 시즌7,NaN,"헤이지니,럭키강이",NaN,키즈,기타,키즈,11분분,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...
3,엉뚱남매 요리조리쇼,NaN,"기원,예현,홍윤화",NaN,키즈,기타,키즈,11분분,먹을 수 있는 물병. 지구 환경을 지키자는 엉뚱남매. 하지만 말처럼 쉽게 지켜지지가...
4,라바 키즈동요 Part3,NaN,-,NaN,키즈,기타,키즈,2분분,CLEAN UP SONG. Clean up Clean up everybody cle...
...,...,...,...,...,...,...,...,...,...
4736,엄마가 화났다,NaN,-,NaN,키즈,학습,키즈,8분분,인기작가 최숙희의 그림책. 아이들에게 건네는 화해와 위로의 메시지! 아이와 엄마 사...
4737,핑크퐁 자동차 동화,NaN,핑크퐁,NaN,키즈,학습,키즈,4분분,Lil Reds First Fire(용감한 소방차 뚜뚜와 뿌뿌). 불이 난 곳이면 ...
4738,간질간질,NaN,-,NaN,키즈,학습,키즈,6분분,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...
4739,월간 아기상어,NaN,아기상어,NaN,키즈,학습,키즈,1분분,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...


### 확인

In [127]:
vod_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4741 entries, 0 to 4740
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           4741 non-null   object
 1   Summary        0 non-null      object
 2   actors         4740 non-null   object
 3   director       0 non-null      object
 4   BigCategory    4741 non-null   object
 5   SmallCategory  4741 non-null   object
 6   Category       4741 non-null   object
 7   runningtime    4741 non-null   object
 8   SMRY           4740 non-null   object
dtypes: object(9)
memory usage: 333.5+ KB


In [128]:
# '-', '[]', '.'이 들어간 값 확인
filtered_rows = vod_data[vod_data['name'].str.contains('\-')]
filtered_rows = vod_data[vod_data['name'].str.contains('\[\]')]
end = vod_data['name'].str.endswith('.')
end_giho = vod_data[end]
end_giho

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY


In [129]:
vod_data.to_csv('../data/vod_all.csv', index=False, encoding='cp949')